Una vez realizado el Parser, tenemos disponibles los archivos .pkl, en el bucket, que contiene las descripciones limpias de cada uno de los inmuebles del dataset.

Como primer paso, descargamos dichos archivos.

In [1]:
project_id = 'cryptic-opus-335323'
bucket_name = 'bdm-unlu'
from google.colab import auth
auth.authenticate_user()
!rm -r html_description_parser
!gsutil cp -r gs://{bucket_name}/html_description_parser/ .

rm: cannot remove 'html_description_parser': No such file or directory
Copying gs://bdm-unlu/html_description_parser/dataset.csv...
Copying gs://bdm-unlu/html_description_parser/parsed_result_0.pkl...
Copying gs://bdm-unlu/html_description_parser/parsed_result_1.pkl...
Copying gs://bdm-unlu/html_description_parser/parsed_result_10.pkl...
- [4 files][ 41.9 MiB/ 41.9 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://bdm-unlu/html_description_parser/parsed_result_11.pkl...
Copying gs://bdm-unlu/html_description_parser/parsed_result_2.pkl...
Copying gs://bdm-unlu/html_description_parser/parsed_result_3.pkl...
Copying gs://bdm-unlu/html_description_parser/parsed_result_4.pkl...
Copying gs://bdm-unlu/html_description_parser/pars

Los unimos todos en un archivo results.pkl.gz. Hardcodeamos los nombres de los archivos, para que siguan el mismo orden de la numeración, dado que esto nos facilitará el trabajo luego.

In [2]:
!cat html_description_parser/parsed_result_0.pkl html_description_parser/parsed_result_1.pkl html_description_parser/parsed_result_2.pkl html_description_parser/parsed_result_3.pkl html_description_parser/parsed_result_4.pkl html_description_parser/parsed_result_5.pkl html_description_parser/parsed_result_6.pkl html_description_parser/parsed_result_7.pkl html_description_parser/parsed_result_8.pkl html_description_parser/parsed_result_9.pkl html_description_parser/parsed_result_10.pkl html_description_parser/parsed_result_11.pkl > html_description_parser/results.pkl.gz

Descomprimimos el archivo.

In [3]:
!gzip -d html_description_parser/results.pkl.gz

Leemos hasta que no tenga mas nada disponible para cargar a los raw_results.

Vemos que leyó 12 archivos.

In [4]:
import pickle

open_file_read = open("html_description_parser/results.pkl", "rb")

raw_results_count = 0
has_load = True
results = []

while has_load:     
  try:
    raw_result = pickle.load(open_file_read)
    raw_results_count += 1
    results.extend(raw_result)
  except Exception as e:
    has_load = False

print("Read {} results.".format(raw_results_count))

Read 12 results.


Descargamos el dataset, para verificar que la cantidad de descripciónes que tenemos, coincide con la cantidad de registros del dataset.

In [5]:
!gsutil cp gs://{bucket_name}/html_spec_table_processor/dataset.csv .

import pandas as pd
df = pd.read_csv("dataset.csv")

Copying gs://bdm-unlu/html_spec_table_processor/dataset.csv...
/ [1 files][ 33.2 MiB/ 33.2 MiB]                                                
Operation completed over 1 objects/33.2 MiB.                                     


Efectivamente vemos que si, ya que la resta de estas dos cantidades nos da 0.

In [6]:
len(df["permalink"]) - len(results)

0

Nos fijamos si el orden que siguen las descripciones, es el mismo orden que tienen los registros del dataset.

Para esto, primero hacemos una funcion que nos permita extraer de la lista de resultados, que tiene la forma [permalink, descripción], la lista de permalinks de estos resultados.

Comparamos la lista de permalinks del dataset, contra la lista de permalinks obtenida anteriormente.

Esto nos devolverá una lista que contendrá unicamente valores verdadero o falso. Si realizamos la suma de esta lista, nos devolverá la cantidad de valores verdadero que hay en total.

Si restamos esta cantidad, con la cantidad de permalinks del dataset, nos dará la cantidad de permalinks que no están en el mismo orden, y vemos que efectivamente es 0.

In [21]:
def column(matrix, i):
    return [row[i] for row in matrix]
permalinks = column(results, 0)

equal_permalinks_amount = sum(df["permalink"] == permalinks)

equal_permalinks_amount - len(df["permalink"])


0

Realizamos una función que verifica, dada una descripción, si encuentra una ocurrencia similar a las palabras PILETA o PISCINA, haciendo uso del metodo get_close_matches, de la librería difflib. En caso de encontrar esta similitud, devuelve el valor True.

In [ ]:
import difflib
def has_pool(cleaned_description):
  has_pool = False

  
  pileta_matches = difflib.get_close_matches('PILETA', cleaned_description, cutoff=0.8)
  if len(pileta_matches) > 0:
    has_pool = True
  else:
    piscina_matches = difflib.get_close_matches('PISCINA', cleaned_description, cutoff=0.8)
    has_pool = len(piscina_matches) > 0
  
  return has_pool

Otra función, igual a la anterior, pero encontrando las palabras TERRAZA o BALCÓN.

In [ ]:
def has_terrace(cleaned_description):
  has_terrace = False

  
  terraza_matches = difflib.get_close_matches('TERRAZA', cleaned_description, cutoff=0.8)
  if len(terraza_matches) > 0:
    has_terrace = True
  else:
    balcon_matches = difflib.get_close_matches('BALCÓN', cleaned_description, cutoff=0.8)
    has_terrace = len(balcon_matches) > 0
  
  return has_terrace

Similar a la funcion anterior, pero verifica la palabra JACUZZI.

In [ ]:
def has_jacuzzi(cleaned_description):
  terraza_matches = difflib.get_close_matches('JACUZZI', cleaned_description, cutoff=0.8)
  
  return len(terraza_matches) > 0

En caso de esta función, verifica la existencia de la palabra LAVADERO.

In [ ]:
def has_washer(cleaned_description):
  has_washer = difflib.get_close_matches('LAVADERO', cleaned_description, cutoff=0.8)
  
  return len(has_washer) > 0

Creamos listas para cada una de las funciones que realizamos, y agregamos a la lista, para cada una de las descripcciones, que retornan dichas funciones.

In [ ]:
has_pool_list = []
has_terrace_list = []
has_jacuzzi_list = []
has_washer_list = []
for result in results:
  has_pool_list.append(has_pool(result[1]))
  has_terrace_list.append(has_terrace(result[1]))
  has_jacuzzi_list.append(has_jacuzzi(result[1]))
  has_washer_list.append(has_washer(result[1]))
  

De todos los inmuebles, esta cantidad, pudimos determinar que tienen pileta.

In [ ]:
sum(has_pool_list)

18622

Esta cantidad, tienen terraza.

In [ ]:
sum(has_terrace_list)

51302

Por otro lado, estos tienen jacuzzi.

In [ ]:
sum(has_jacuzzi_list)

2268

Esta otra cantidad, tiene lavadero.

In [ ]:
sum(has_washer_list)

23167

Agregamos al dataset estas columnas.

In [ ]:
df["has_pool"] = has_pool_list
df["has_terrace"] = has_terrace_list
df["has_jacuzzi"] = has_jacuzzi_list
df["has_washer"] = has_washer_list

Vemos como queda la lista resultante de columnas.

Además, podemos observar que las primeras 3 columnas tienen un valor erroneo, vamos a eliminarlo.

In [ ]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'title', 'permalink',
       'city', 'state', 'latitude', 'longitude', 'neighborhood', 'seller_id',
       'seller_city', 'seller_state', 'real_estate_agency',
       'seller_cancelations', 'seller_claims', 'seller_handling_time',
       'seller_sales', 'currency_id', 'price', 'item_condition',
       'has_air_conditioning', 'has_telephone_line', 'bedrooms',
       'covered_area', 'full_bathrooms', 'rooms', 'total_area', 'operation',
       'property_type', 'with_virtual_tour', 'Superficie total',
       'Superficie cubierta', 'Ambientes', 'Dormitorios', 'Baños', 'Cocheras',
       'Cantidad de pisos', 'Disposición', 'Antigüedad', 'Expensas',
       'Departamentos por piso', 'Número de piso de la unidad', 'Orientación',
       'Tipo de departamento', 'Bodegas', 'Tipo de casa',
       'Superficie de terreno', 'has_pool', 'has_terrace', 'has_jacuzzi',
       'has_washer'],
      dtype='object')

Dropeamos estas columnas.

In [ ]:
df = df.drop(df.columns[0], axis=1)
df = df.drop(df.columns[0], axis=1)
df = df.drop(df.columns[0], axis=1)

Vemos como queda el dataset.

In [ ]:
df[:2]

,title,permalink,city,state,latitude,longitude,neighborhood,seller_id,seller_city,seller_state,real_estate_agency,seller_cancelations,seller_claims,seller_handling_time,seller_sales,currency_id,price,item_condition,has_air_conditioning,has_telephone_line,bedrooms,covered_area,full_bathrooms,rooms,total_area,operation,property_type,with_virtual_tour,Superficie total,Superficie cubierta,Ambientes,Dormitorios,Baños,Cocheras,Cantidad de pisos,Disposición,Antigüedad,Expensas,Departamentos por piso,Número de piso de la unidad,Orientación,Tipo de departamento,Bodegas,Tipo de casa,Superficie de terreno,has_pool,has_terrace,has_jacuzzi,has_washer
0,Departamento Venta 3 Ambientes Palermo Amenit...,https://departamento.mercadolibre.com.ar/MLA-1...,Capital Federal,Capital Federal,-34.579563,-58.433160,Palermo Hollywood,314431007,Palermo,Capital Federal,True,0.0,0.0,0.0,0.0,USD,270000.0,Usado,No,No,2.0,85 m²,3.0,3.0,85 m²,Venta,Departamento,No,85 m²,85 m²,3.0,2.0,3.0,1.0,7.0,Frente,10 años,20000 ARS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,False,False
1,Venta Departamento 3 Ambientes Piso 15 Palermo,https://departamento.mercadolibre.com.ar/MLA-1...,Capital Federal,Capital Federal,-34.573490,-58.436577,Palermo Hollywood,466036843,Monserrat,Capital Federal,True,0.0,0.0,0.0,0.0,USD,148500.0,Usado,NaN,NaN,2.0,46 m²,1.0,3.0,55 m²,Venta,Departamento,No,55 m²,46 m²,3.0,2.0,1.0,NaN,NaN,NaN,50 años,12.5 ARS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False,False


In [ ]:
df.to_csv("dataset.csv")

In [ ]:
!gsutil cp dataset.csv gs://{bucket_name}/html_description_parser/

Copying file://dataset.csv [Content-Type=text/csv]...
-
Operation completed over 1 objects/34.9 MiB.                                     


Queda pendiente hacer un Word Cloud, para ver cuales son las palabras mas frecuentes de la descripción, y determinar de esta forma, graficamente, si podemos agregar alguna otra funcion, para enriquecer el dataset.

Pero dado que requiere un costo computacional alto, postergamos su realización.

In [ ]:
#PENDIENTE. Costo computacional muy alto

#text = ''
#for result in results[:1000]:
#  for word in result[1]:
#    text = text + ' ' + word

# Import package
#import matplotlib.pyplot as plt# Define a function to plot word cloud
#def plot_cloud(wordcloud):
    # Set figure size
#    plt.figure(figsize=(40, 30))
    # Display image
#    plt.imshow(wordcloud) 
    # No axis details
#    plt.axis("off");

# Import package
#from wordcloud import WordCloud, STOPWORDS# Generate word cloud
#wordcloud = WordCloud(width= 3000, height = 2000, random_state=1, background_color='salmon', colormap='Pastel1', collocations=False, stopwords = STOPWORDS).generate(text)# Plot
#plot_cloud(wordcloud)